In [1]:
!pip install --quiet arcgis

In [2]:
import os
from arcgis.gis import GIS;
import pandas as pd;
import requests;

In [3]:
pd_orgid = pd.DataFrame(columns = [
     "stateID"
    ,"organizationIdentifier"
]);

pd_assmnts = pd.DataFrame(columns = [
     "organizationIdentifier"
    ,"assessmentUnitIdentifier"
    ,"reportingCycle"
    ,"agencyCode"
    ,"trophicStatusCode"
    ,"epaIRCategory"
]);

pd_uses = pd.DataFrame(columns = [
     "organizationIdentifier"
    ,"assessmentUnitIdentifier"
    ,"reportingCycle"
    ,"useName"
    ,"useAttainmentCode"
    ,"threatenedIndicator"
    ,"trendCode"
    ,"agencyCode"
]);

pd_parms = pd.DataFrame(columns = [
     "organizationIdentifier"
    ,"assessmentUnitIdentifier"
    ,"reportingCycle"
    ,"parameterStatusName"
    ,"parameterName"
    ,"pollutantIndicator"
]);

pd_parms2use = pd.DataFrame(columns = [
     "organizationIdentifier"
    ,"assessmentUnitIdentifier"
    ,"reportingCycle"
    ,"parameterName"
    ,"associatedUseName"
    ,"parameterAttainmentCode"
    ,"trendCode"
]);

pd_srcs = pd.DataFrame(columns = [
     "organizationIdentifier"
    ,"assessmentUnitIdentifier"
    ,"reportingCycle"
    ,"sourceName"
    ,"sourceConfirmedIndicator"
]);

pd_src2caus = pd.DataFrame(columns = [
     "organizationIdentifier"
    ,"assessmentUnitIdentifier"
    ,"reportingCycle"
    ,"sourceName"
    ,"causeName"
]);

pd_docs = pd.DataFrame(columns = [
     "organizationIdentifier"
    ,"assessmentUnitIdentifier"
    ,"reportingCycle"
    ,"documentName"
    ,"documentFileName"
    ,"documentURL"
    ,"agencyCode"
]);


In [4]:
# Pull the list of organization identifiers
org_url = r"https://attains.epa.gov/attains-public/api/domains";
response = requests.get(
     org_url
    ,params={
         'domainName': 'OrgStateCode'
    }
);
resp_json = response.json();

orgs = [];
for item in resp_json:
    
    if item['context'] in ("EPA","CA_BVR","NNEPA") or item['context'][:5] == "TEST_":
        continue;

    orgs.append({
         "stateID": item['code']
        ,"organizationIdentifier": item['context']
    });
    
    
pd_orgid = pd_orgid.head(0)                                 \
    .append(orgs)                                           \
    .sort_values(by=['stateID','organizationIdentifier']);
    
len(pd_orgid)

69

In [ ]:
# For each orgid pull the assessments into a table
ste_url = r"https://attains.epa.gov/attains-public/api/assessments"

for index, row in pd_orgid.iterrows():
    
    response = requests.get(
         ste_url
        ,params = {
             "organizationId": row['organizationIdentifier']
        }
    );
    resp_json = response.json();
    print(row['organizationIdentifier'] + " " + str(len(resp_json['items'][0]['assessments'])));
    cycle_year = resp_json['items'][0]['reportingCycleText'];
    
    assmnt   = [];
    uses     = [];
    parms    = [];
    parm2use = [];
    srcs     = [];
    src2caus = [];
    docs     = [];

    for item in resp_json['items'][0]['assessments']:
        assmnt.append({
             "organizationIdentifier": row['organizationIdentifier']
            ,"assessmentUnitIdentifier": item['assessmentUnitIdentifier']
            ,'reportingCycle': cycle_year
            ,"agencyCode": item['agencyCode']
            ,"trophicStatusCode": item['trophicStatusCode']
            ,"epaIRCategory": item['epaIRCategory']
        });
        
        if item['useAttainments'] is not None and len(item['useAttainments']) > 0:
            for uitem in item['useAttainments']:
                uses.append({
                     "organizationIdentifier": row['organizationIdentifier']
                    ,"assessmentUnitIdentifier": item['assessmentUnitIdentifier']
                    ,"reportingCycle": cycle_year
                    ,"useName": uitem['useName']
                    ,"useAttainmentCode": uitem['useAttainmentCode']
                    ,"threatenedIndicator": uitem['threatenedIndicator']
                    ,"trendCode": uitem['trendCode']
                    ,"agencyCode": uitem['agencyCode']
                });
                
        if item['parameters'] is not None and len(item['parameters']) > 0:
            for pitem in item['parameters']:
                parms.append({
                     "organizationIdentifier": row['organizationIdentifier']
                    ,"assessmentUnitIdentifier": item['assessmentUnitIdentifier']
                    ,"reportingCycle": cycle_year
                    ,"parameterStatusName": pitem['parameterStatusName']
                    ,"parameterName": pitem['parameterName']
                    ,"pollutantIndicator": pitem['pollutantIndicator']
                });
                
                if pitem['associatedUses'] is not None and len(pitem['associatedUses']) > 0:
                    for auitem in pitem['associatedUses']:
                        parm2use.append({
                             "organizationIdentifier": row['organizationIdentifier']
                            ,"assessmentUnitIdentifier": item['assessmentUnitIdentifier']
                            ,"reportingCycle": cycle_year
                            ,"parameterName": pitem['parameterName']
                            ,"associatedUseName": auitem['associatedUseName']
                            ,"parameterAttainmentCode": auitem['parameterAttainmentCode']
                            ,"trendCode": auitem['trendCode']
                        });
                        
        if item['probableSources'] is not None and len(item['probableSources']) > 0:
            for psitem in item['probableSources']:
                srcs.append({
                     "organizationIdentifier": row['organizationIdentifier']
                    ,"assessmentUnitIdentifier": item['assessmentUnitIdentifier']
                    ,"reportingCycle": cycle_year
                    ,"sourceName": psitem['sourceName']
                    ,"sourceConfirmedIndicator": psitem['sourceConfirmedIndicator']
                });
                
                if psitem['associatedCauseNames'] is not None and len(psitem['associatedCauseNames']) > 0:
                    for acitem in psitem['associatedCauseNames']:
                        src2caus.append({
                             "organizationIdentifier": row['organizationIdentifier']
                            ,"assessmentUnitIdentifier": item['assessmentUnitIdentifier']
                            ,"reportingCycle": cycle_year
                            ,"sourceName": psitem['sourceName']
                            ,"causeName": acitem['causeName']
                        });
                        
        if item['documents'] is not None and len(item['documents']) > 0:
            for docitem in item['documents']:
                docs.append({
                     "organizationIdentifier": row['organizationIdentifier']
                    ,"assessmentUnitIdentifier": item['assessmentUnitIdentifier']
                    ,"reportingCycle": cycle_year
                    ,"documentName": docitem['documentName']
                    ,"documentFileName": docitem['documentFileName']
                    ,"documentURL": docitem['documentURL']
                    ,"agencyCode": docitem['agencyCode']
                });
                
    if len(assmnt) > 0:
        pd_assmnts  = pd_assmnts.append(assmnt);
    if len(uses) > 0:
        pd_uses     = pd_uses.append(uses);
    if len(parms) > 0:
        pd_parms    = pd_parms.append(parms);
    if len(parm2use) > 0:
        pd_parm2use = pd_parms2use.append(parm2use);
    if len(srcs) > 0:
        pd_srcs     = pd_srcs.append(srcs);
    if len(src2caus) > 0:
        pd_src2caus = pd_src2caus.append(src2caus);
    if len(docs) > 0:
        pd_docs     = pd_docs.append(docs);
    
    #if index > 1:
    #    break
            
with pd.ExcelWriter('output.xlsx') as writer:
    pd_assmnts.to_excel(writer, sheet_name='Assessments');
    pd_uses.to_excel(writer, sheet_name='Uses');
    pd_parms.to_excel(writer, sheet_name='Parameters');
    pd_parm2use.to_excel(writer, sheet_name='Parms2Uses');
    pd_srcs.to_excel(writer, sheet_name='Sources');
    pd_src2caus.to_excel(writer, sheet_name='Sources2Causes');
    pd_docs.to_excel(writer, sheet_name='Documents');
    

AKDECWQ 462
21AWIC 1814
ARDEQH2O 867
21AS 86
21ARIZ 356
CA_SWRCB 2698
HVTEPA 30
21COL001 1490
CT_DEP01 1494
DOEE 36
21DELAWQ 379
21FL303D 6543
21GAEPD 2616
21GUAM 333
21HI 756
21IOWA 1378
IDEQ 5766
IL_EPA 3738
21IND 16283
21KAN001 2302
21KY 2751
LADEQWPD 545
MA_DEP 2182
MDE_EASP 796
MEDEP 1159
21MICH 7654
FONDULAC 31
MNPCA 24437
MDNR 4320
21AQ 84
21MSWQ 1030
MTDEQ 1194
21NC01WQ 3705
21NDHDWQ 1696
21NEB001 2118
11113300 32441
21NJDEP1 958
21NMEX 839
PUEBLOOFTESUQUE 8
PUEBLO_POJOAQUE 3
SANILDEFONSODECP 13
21NEV1 660
21NYDECA 4807
21OHIO 1583
CHOCNAT 19
CNENVSER 20
CPNWATER 14
DELAWARENATION 13
KICKAPOO 8
OKDEQ 4226
O_MTRIBE 6
SCEQ 9
SFNOES 14
OREGONDEQ 2204
